In [9]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
def extract_data_egmont(url):
    
    """
    This function will collect the grant information provided from the egmont foundation grant database with the url: "https://www.egmont.dk/stoettede-indsatser"
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    receiver_l = []
    description_l = []
    year_l = []
    category_l = []
    amount_L = []
    title_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url ("https://www.egmont.dk/stoettede-indsatser")
    return: Dataframe with text extracted
    
    """

    
    receiver_l = []
    description_l = []
    year_l = []
    category_l = []
    amount_L = []
    title_l = []
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    #click on the #donations button
    #action = ActionChains(driver)
    time.sleep(10)
    
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child(1) > button > div.grant-accordion__trigger-title")))
    action = ActionChains(driver)
    action.move_to_element(element).perform()
    
    #element_mover.move_to_element.perform()
    time.sleep(5)
    try:
        for page_num in range(1,19): #cliscking next page 19 times
            time.sleep(3)
            for i in range(1,11):
                try:
                    action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button")).perform()  #moving to the line of interest
                    element2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button")))
                    element2.click() 
                except StaleElementReferenceException:
                    time.sleep(10)
                    print(i)
                    action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")).perform()  #moving to the line of interes
                    element2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")))
                    element2.click() 
                except ElementClickInterceptedException:
                    time.sleep(5)
                    print(i)
                    action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")).perform()  #moving to the line of interes
                    element2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")))
                    element2.click() 
                except NoSuchElementException: #avoid programme crashing
                    time.sleep(5)
                    print(i)
                    action.move_to_element(driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")).perform()  #moving to the line of interes
                    element2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button > div.grant-accordion__trigger-title")))
                    element2.click()
                title = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > button").text #grant title
                receiver = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > div.grant-accordion__info.row.grant-accordion__info--active > div:nth-child(1)").text #receiver
                category = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > div.grant-accordion__info.row.grant-accordion__info--active > div:nth-child(2)").text # cagtegory
                description = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > div.grant-accordion__content--active.grant-accordion__content > div").text #description
                year = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > div.grant-accordion__info.row.grant-accordion__info--active > div:nth-child(3)").text #year
                amount = driver.find_element(By.CSS_SELECTOR, f"#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div:nth-child(3) > div:nth-child({i}) > div.grant-accordion__info.row.grant-accordion__info--active > div.col-sm-2.col-md-3").text # amount
                

                #append
                receiver_l.append(receiver)
                description_l.append(description)
                year_l.append(year)
                category_l.append(category)
                amount_L.append(amount)
                title_l.append(title)
            
            ActionChains(driver).move_to_element(driver.find_element(By.CSS_SELECTOR, "#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div.pager.grant-accordion__pager > button.pager__page.pager__page--next")).perform()  #move to click button
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#__nuxt > div.app > main > div > div.layout-builder > div.section.layout_onecol.section--spacing-top-small.section--spacing-bottom-small > div > div > div > div > div > div.pager.grant-accordion__pager > button.pager__page.pager__page--next")))
            element.click()

                
            
    except NoSuchElementException: #avoid programme crashing
        print("Loop ended")

 

    df_final = pd.DataFrame(list(zip(year_l, receiver_l, amount_L, description_l, category_l, title_l)), columns=["Year",'Receiver','Grant size (DKK)', 'Description', "Project Category", "Project Title"])

    return df_final

biglist = extract_data_egmont("https://www.egmont.dk/stoettede-indsatser")
pd.set_option('display.max_columns', None)
biglist.to_csv('egmont_data.csv', index=True)

5
Loop ended
